In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

view_log_train = pd.read_csv('../data/view_log.csv')
article_info = pd.read_csv('../data/article_info.csv')
submission = pd.read_csv('../submission/sample_submission.csv')

In [5]:
view_log_train.head()

,userID,articleID,userRegion,userCountry
0,USER_0000,ARTICLE_0661,NY,US
1,USER_0000,ARTICLE_2316,NY,US
2,USER_0000,ARTICLE_1345,NY,US
3,USER_0000,ARTICLE_1089,NY,US
4,USER_0000,ARTICLE_1484,NY,US


In [6]:
article_info.head()

,articleID,Title,Content,Format,Language,userID,userCountry,userRegion
0,ARTICLE_0000,19 Tips For Everyday Git Use,I've been using git full time for the past 4 y...,HTML,en,USER_0683,NaN,NaN
1,ARTICLE_0001,Intel buys computer vision startup Itseez to i...,Intel has acquired computer vision and machine...,HTML,en,USER_1129,NaN,NaN
2,ARTICLE_0002,Practical End-to-End Testing with Protractor,One of the reasons AngularJS is so great to wo...,HTML,en,USER_0256,NaN,NaN
3,ARTICLE_0003,Corporate venture growth in Brazil is another ...,Despite recent positive news and a renewed int...,HTML,en,USER_1304,NaN,NaN
4,ARTICLE_0004,Cross-channel user experiences with Drupal (aw...,"Last year around this time, I wrote that The B...",HTML,en,USER_0336,NaN,NaN


In [2]:
user_article_matrix = view_log_train.groupby(['userID', 'articleID']).size().unstack(fill_value=0)

In [4]:
user_article_matrix.head()

articleID,ARTICLE_0000,ARTICLE_0001,ARTICLE_0002,ARTICLE_0003,ARTICLE_0004,ARTICLE_0005,ARTICLE_0006,ARTICLE_0007,ARTICLE_0008,ARTICLE_0009,...,ARTICLE_2998,ARTICLE_2999,ARTICLE_3000,ARTICLE_3001,ARTICLE_3002,ARTICLE_3003,ARTICLE_3004,ARTICLE_3005,ARTICLE_3006,ARTICLE_3007
userID,,,,,,,,,,,,,,,,,,,,,
USER_0000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
USER_0001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
USER_0002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
USER_0003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
USER_0004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
user_similarity = cosine_similarity(user_article_matrix)


In [8]:
user_similarity

array([[1.        , 0.        , 0.        , ..., 0.02571722, 0.        ,
        0.01028689],
       [0.        , 1.        , 0.        , ..., 0.        , 0.00847884,
        0.02581989],
       [0.        , 0.        , 1.        , ..., 0.        , 0.06495046,
        0.        ],
       ...,
       [0.02571722, 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.00847884, 0.06495046, ..., 0.        , 1.        ,
        0.        ],
       [0.01028689, 0.02581989, 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [10]:
user_predicted_scores = user_similarity.dot(user_article_matrix) / np.array([np.abs(user_similarity).sum(axis=1)]).T


In [11]:
user_predicted_scores

array([[0.01635677, 0.03922991, 0.00452033, ..., 0.00937059, 0.02394942,
        0.02622418],
       [0.01945815, 0.00598531, 0.0004365 , ..., 0.01538668, 0.02710833,
        0.00398137],
       [0.03393534, 0.01010633, 0.00108347, ..., 0.06788127, 0.03008129,
        0.0098929 ],
       ...,
       [0.01438303, 0.00844972, 0.00021909, ..., 0.03515472, 0.08179751,
        0.00730041],
       [0.0373851 , 0.00141859, 0.00021209, ..., 0.02351579, 0.06250373,
        0.00406303],
       [0.01883932, 0.02046381, 0.00180259, ..., 0.01480964, 0.02338439,
        0.02576106]])

In [12]:
recommendations = []
for idx, user in enumerate(user_article_matrix.index):
    sorted_indices = user_predicted_scores[idx].argsort()[::-1]
    top5recommend = [article for article in user_article_matrix.columns[sorted_indices]][:5]

    for article in top5recommend:
        recommendations.append([user, article])


In [13]:
top_recommendations = pd.DataFrame(recommendations, columns=['userID', 'articleID'])

submission['articleID'] = top_recommendations['articleID']

submission.to_csv('../submission/baseline_submission.csv', index=False)


In [14]:
submission

,userID,articleID
0,USER_0000,ARTICLE_0411
1,USER_0000,ARTICLE_0664
2,USER_0000,ARTICLE_1568
3,USER_0000,ARTICLE_1230
4,USER_0000,ARTICLE_2255
...,...,...
7070,USER_1420,ARTICLE_0030
7071,USER_1420,ARTICLE_0614
7072,USER_1420,ARTICLE_1901
7073,USER_1420,ARTICLE_0714
